In [0]:
# bibliotecas utilizadas
import pandas as pd
import unicodedata
import re

## Etapa 1 - Coleta dos dados

In [0]:
#Ver o local que está o arquivo csv
display(dbutils.fs.ls('dbfs:/FileStore/'))

path,name,size,modificationTime
dbfs:/FileStore/Consulta_de_Contratos_Públicos.csv,Consulta_de_Contratos_Públicos.csv,363590,1743478130000
dbfs:/FileStore/State_of_data_BR_2023_Kaggle___df_survey_2023.csv,State_of_data_BR_2023_Kaggle___df_survey_2023.csv,15241470,1744069381000
dbfs:/FileStore/tables/,tables/,0,0


In [0]:
# Carregando os dados do dbfs no dataframe para visualização dos dados
df = spark.read.csv('dbfs:/FileStore/State_of_data_BR_2023_Kaggle___df_survey_2023.csv', header=True, sep=',', inferSchema=True)
df.limit(5).display()

"('P0', 'id')","('P1_a ', 'Idade')","('P1_a_1 ', 'Faixa idade')","('P1_b ', 'Genero')","('P1_c ', 'Cor/raca/etnia')","('P1_d ', 'PCD')","('P1_e ', 'experiencia_profissional_prejudicada')","('P1_e_1 ', 'Não acredito que minha experiência profissional seja afetada')","('P1_e_2 ', 'Experiencia prejudicada devido a minha Cor Raça Etnia')","('P1_e_3 ', 'Experiencia prejudicada devido a minha identidade de gênero')","('P1_e_4 ', 'Experiencia prejudicada devido ao fato de ser PCD')","('P1_f ', 'aspectos_prejudicados')","('P1_f_1', 'Quantidade de oportunidades de emprego/vagas recebidas')","('P1_f_2', 'Senioridade das vagas recebidas em relação à sua experiência')","('P1_f_3', 'Aprovação em processos seletivos/entrevistas')","('P1_f_4', 'Oportunidades de progressão de carreira')","('P1_f_5', 'Velocidade de progressão de carreira')","('P1_f_6', 'Nível de cobrança no trabalho/Stress no trabalho')","('P1_f_7', 'Atenção dada diante das minhas opiniões e ideias')","('P1_f_8', 'Relação com outros membros da empresa, em momentos de trabalho')","('P1_f_9', 'Relação com outros membros da empresa, em momentos de integração e outros momentos fora do trabalho')","('P1_g ', 'vive_no_brasil')","('P1_i ', 'Estado onde mora')","('P1_i_1 ', 'uf onde mora')","('P1_i_2 ', 'Regiao onde mora')","('P1_j ', 'Mudou de Estado?')","('P1_k ', 'Regiao de origem')","('P1_l ', 'Nivel de Ensino')","('P1_m ', 'Área de Formação')","('P2_a ', 'Qual sua situação atual de trabalho?')","('P2_b ', 'Setor')","('P2_c ', 'Numero de Funcionarios')","('P2_d ', 'Gestor?')","('P2_e ', 'Cargo como Gestor')","('P2_f ', 'Cargo Atual')","('P2_g ', 'Nivel')","('P2_h ', 'Faixa salarial')","('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')","('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')","('P2_k ', 'Você está satisfeito na sua empresa atual?')","('P2_l ', 'Qual o principal motivo da sua insatisfação com a empresa atual?')","('P2_l_1 ', 'Falta de oportunidade de crescimento no emprego atual')","('P2_l_2 ', 'Salário atual não corresponde ao mercado')","('P2_l_3 ', 'Não tenho uma boa relação com meu líder/gestor')","('P2_l_4 ', 'Gostaria de trabalhar em em outra área de atuação')","('P2_l_5 ', 'Gostaria de receber mais benefícios')","('P2_l_6 ', 'O clima de trabalho/ambiente não é bom')","('P2_l_7 ', 'Falta de maturidade analítica na empresa')","('P2_m ', 'Você participou de entrevistas de emprego nos últimos 6 meses?')","('P2_n ', 'Você pretende mudar de emprego nos próximos 6 meses?')","('P2_o ', 'Quais os principais critérios que você leva em consideração no momento de decidir onde trabalhar?')","('P2_o_1 ', 'Remuneração/Salário')","('P2_o_2 ', 'Benefícios')","('P2_o_3 ', 'Propósito do trabalho e da empresa')","('P2_o_4 ', 'Flexibilidade de trabalho remoto')","('P2_o_5 ', 'Ambiente e clima de trabalho')","('P2_o_6 ', 'Oportunidade de aprendizado e trabalhar com referências na área')","('P2_o_7 ', 'Plano de carreira e oportunidades de crescimento profissional')","('P2_o_8 ', 'Maturidade da empresa em termos de tecnologia e dados')","('P2_o_9 ', 'Qualidade dos gestores e líderes')","('P2_o_10 ', 'Reputação que a empresa tem no mercado')","('P2_q ', 'Empresa que trabaha passou por layoff em 2023')","('P2_r ', 'Atualmente qual a sua forma de trabalho?')","('P2_s ', 'Qual a forma de trabalho ideal para você?')","('P2_t ', 'Caso sua empresa decida pelo modelo 100% presencial qual será sua atitude?')","('P3_a ', 'Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?')","('P3_b ', 'Quais desses papéis/cargos fazem parte do time (ou chapter) de dados da sua empresa?')","('P3_b_1 ', 'Analytics Engineer')","('P3_b_2 ', 'Engenharia de Dados/Data Engineer')","('P3_b_3 ', 'Analista de Dados/Data Analyst')","('P3_b_4 ', 'Cientista de Dados/Data Scientist')","('P3_b_5 ', 'Database Administrator/DBA')","('P3_b_6 ', 'Analista de Business Intelligence/BI')","('P3_b_7 ', 'Arqui

## Etapa 2 - Camada Silver

In [0]:
import re, unicodedata

# Função para normalizar o nome das colunas:
# - Remove acentos, substitui caracteres não alfanuméricos por underscore, remove underscores duplicados e de inicio e fim, converte para caixa alta
def normalize_column(col_name):
    normalized = unicodedata.normalize('NFKD', col_name).encode("ASCII", "ignore").decode("utf-8")
    normalized = re.sub(r'[^\w]', '_', normalized)
    normalized = re.sub(r'_+', '_', normalized).strip('_')
    return normalized.upper()

# Le o CSV bruto com header e inferSchema
df_raw = spark.read.csv(
    "dbfs:/FileStore/State_of_data_BR_2023_Kaggle___df_survey_2023.csv", 
    header=True, 
    inferSchema=True, 
    sep=","
)

# Exibir o schema original para conferência
df_raw.printSchema()


root
 |-- ('P0', 'id'): string (nullable = true)
 |-- ('P1_a ', 'Idade'): integer (nullable = true)
 |-- ('P1_a_1 ', 'Faixa idade'): string (nullable = true)
 |-- ('P1_b ', 'Genero'): string (nullable = true)
 |-- ('P1_c ', 'Cor/raca/etnia'): string (nullable = true)
 |-- ('P1_d ', 'PCD'): string (nullable = true)
 |-- ('P1_e ', 'experiencia_profissional_prejudicada'): string (nullable = true)
 |-- ('P1_e_1 ', 'Não acredito que minha experiência profissional seja afetada'): integer (nullable = true)
 |-- ('P1_e_2 ', 'Experiencia prejudicada devido a minha Cor Raça Etnia'): integer (nullable = true)
 |-- ('P1_e_3 ', 'Experiencia prejudicada devido a minha identidade de gênero'): integer (nullable = true)
 |-- ('P1_e_4 ', 'Experiencia prejudicada devido ao fato de ser PCD'): integer (nullable = true)
 |-- ('P1_f ', 'aspectos_prejudicados'): string (nullable = true)
 |-- ('P1_f_1', 'Quantidade de oportunidades de emprego/vagas recebidas'): string (nullable = true)
 |-- ('P1_f_2', 'Seniori

In [0]:
# Aplicar a normalização dos nomes das colunas, isso transformará, por exemplo, "('P1_a ', 'Idade')" em "P1_A_IDADE"
new_columns = [normalize_column(c) for c in df_raw.columns]
df_normalized = df_raw.toDF(*new_columns)

# Exibir os novos nomes de colunas e o schema normalizado para conferência
print("Novos nomes das colunas:")
print(df_normalized.columns)
df_normalized.printSchema()

Novos nomes das colunas:
['P0_ID', 'P1_A_IDADE', 'P1_A_1_FAIXA_IDADE', 'P1_B_GENERO', 'P1_C_COR_RACA_ETNIA', 'P1_D_PCD', 'P1_E_EXPERIENCIA_PROFISSIONAL_PREJUDICADA', 'P1_E_1_NAO_ACREDITO_QUE_MINHA_EXPERIENCIA_PROFISSIONAL_SEJA_AFETADA', 'P1_E_2_EXPERIENCIA_PREJUDICADA_DEVIDO_A_MINHA_COR_RACA_ETNIA', 'P1_E_3_EXPERIENCIA_PREJUDICADA_DEVIDO_A_MINHA_IDENTIDADE_DE_GENERO', 'P1_E_4_EXPERIENCIA_PREJUDICADA_DEVIDO_AO_FATO_DE_SER_PCD', 'P1_F_ASPECTOS_PREJUDICADOS', 'P1_F_1_QUANTIDADE_DE_OPORTUNIDADES_DE_EMPREGO_VAGAS_RECEBIDAS', 'P1_F_2_SENIORIDADE_DAS_VAGAS_RECEBIDAS_EM_RELACAO_A_SUA_EXPERIENCIA', 'P1_F_3_APROVACAO_EM_PROCESSOS_SELETIVOS_ENTREVISTAS', 'P1_F_4_OPORTUNIDADES_DE_PROGRESSAO_DE_CARREIRA', 'P1_F_5_VELOCIDADE_DE_PROGRESSAO_DE_CARREIRA', 'P1_F_6_NIVEL_DE_COBRANCA_NO_TRABALHO_STRESS_NO_TRABALHO', 'P1_F_7_ATENCAO_DADA_DIANTE_DAS_MINHAS_OPINIOES_E_IDEIAS', 'P1_F_8_RELACAO_COM_OUTROS_MEMBROS_DA_EMPRESA_EM_MOMENTOS_DE_TRABALHO', 'P1_F_9_RELACAO_COM_OUTROS_MEMBROS_DA_EMPRESA_EM_MOMENTOS_DE_

In [0]:
# Define o caminho de armazenamento na camada Bronze
bronze_path = "dbfs:/mnt/bronze/state_of_data_2023"

# Gravar os dados normalizados em formato Delta na camada Bronze
df_normalized.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(bronze_path)

# Cria o schema bronze se ainda não existir
spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")

# Registra a tabela Bronze no metastore apontando para o caminho Delta
spark.sql(f"""
  CREATE OR REPLACE TABLE bronze.STATE_OF_DATA_2023 AS
  SELECT * FROM delta.`{bronze_path}`
""")

Out[15]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql -- visualizando os dados com limit 10
select * from bronze.state_of_data_2023 limit 10

P0_id,P1_a_Idade,P1_a_1_Faixa_idade,P1_b_Genero,P1_c_Cor_raca_etnia,P1_d_PCD,P1_e_experiencia_profissional_prejudicada,P1_e_1_Nao_acredito_que_minha_experiencia_profissional_seja_afetada,P1_e_2_Experiencia_prejudicada_devido_a_minha_Cor_Raca_Etnia,P1_e_3_Experiencia_prejudicada_devido_a_minha_identidade_de_genero,P1_e_4_Experiencia_prejudicada_devido_ao_fato_de_ser_PCD,P1_f_aspectos_prejudicados,P1_f_1_Quantidade_de_oportunidades_de_emprego_vagas_recebidas,P1_f_2_Senioridade_das_vagas_recebidas_em_relacao_a_sua_experiencia,P1_f_3_Aprovacao_em_processos_seletivos_entrevistas,P1_f_4_Oportunidades_de_progressao_de_carreira,P1_f_5_Velocidade_de_progressao_de_carreira,P1_f_6_Nivel_de_cobranca_no_trabalho_Stress_no_trabalho,P1_f_7_Atencao_dada_diante_das_minhas_opinioes_e_ideias,P1_f_8_Relacao_com_outros_membros_da_empresa_em_momentos_de_trabalho,P1_f_9_Relacao_com_outros_membros_da_empresa_em_momentos_de_integracao_e_outros_momentos_fora_do_trabalho,P1_g_vive_no_brasil,P1_i_Estado_onde_mora,P1_i_1_uf_onde_mora,P1_i_2_Regiao_onde_mora,P1_j_Mudou_de_Estado,P1_k_Regiao_de_origem,P1_l_Nivel_de_Ensino,P1_m_Area_de_Formacao,P2_a_Qual_sua_situacao_atual_de_trabalho,P2_b_Setor,P2_c_Numero_de_Funcionarios,P2_d_Gestor,P2_e_Cargo_como_Gestor,P2_f_Cargo_Atual,P2_g_Nivel,P2_h_Faixa_salarial,P2_i_Quanto_tempo_de_experiencia_na_area_de_dados_voce_tem,P2_j_Quanto_tempo_de_experiencia_na_area_de_TI_Engenharia_de_Software_voce_teve_antes_de_comecar_a_trabalhar_na_area_de_dados,P2_k_Voce_esta_satisfeito_na_sua_empresa_atual,P2_l_Qual_o_principal_motivo_da_sua_insatisfacao_com_a_empresa_atual,P2_l_1_Falta_de_oportunidade_de_crescimento_no_emprego_atual,P2_l_2_Salario_atual_nao_corresponde_ao_mercado,P2_l_3_Nao_tenho_uma_boa_relacao_com_meu_lider_gestor,P2_l_4_Gostaria_de_trabalhar_em_em_outra_area_de_atuacao,P2_l_5_Gostaria_de_receber_mais_beneficios,P2_l_6_O_clima_de_trabalho_ambiente_nao_e_bom,P2_l_7_Falta_de_maturidade_analitica_na_empresa,P2_m_Voce_participou_de_entrevistas_de_emprego_nos_ultimos_6_meses,P2_n_Voce_pretende_mudar_de_emprego_nos_proximos_6_meses,P2_o_Quais_os_principais_criterios_que_voce_leva_em_consideracao_no_momento_de_decidir_onde_trabalhar,P2_o_1_Remuneracao_Salario,P2_o_2_Beneficios,P2_o_3_Proposito_do_trabalho_e_da_empresa,P2_o_4_Flexibilidade_de_trabalho_remoto,P2_o_5_Ambiente_e_clima_de_trabalho,P2_o_6_Oportunidade_de_aprendizado_e_trabalhar_com_referencias_na_area,P2_o_7_Plano_de_carreira_e_oportunidades_de_crescimento_profissional,P2_o_8_Maturidade_da_empresa_em_termos_de_tecnologia_e_dados,P2_o_9_Qualidade_dos_gestores_e_lideres,P2_o_10_Reputacao_que_a_empresa_tem_no_mercado,P2_q_Empresa_que_trabaha_passou_por_layoff_em_2023,P2_r_Atualmente_qual_a_sua_forma_de_trabalho,P2_s_Qual_a_forma_de_trabalho_ideal_para_voce,P2_t_Caso_sua_empresa_decida_pelo_modelo_100_presencial_qual_sera_sua_atitude,P3_a_Qual_o_numero_aproximado_de_pessoas_que_atuam_com_dados_na_sua_empresa_hoje,P3_b_Quais_desses_papeis_cargos_fazem_parte_do_time_ou_chapter_de_dados_da_sua_empresa,P3_b_1_Analytics_Engineer,P3_b_2_Engenharia_de_Dados_Data_Engineer,P3_b_3_Analista_de_Dados_Data_Analyst,P3_b_4_Cientista_de_Dados_Data_Scientist,P3_b_5_Database_Administrator_DBA,P3_b_6_Analista_de_Business_Intelligence_BI,P3_b_7_Arquiteto_de_Dados_Data_Architect,P3_b_8_Data_Product_Manager_DPM,P3_b_9_Business_Analyst,P3_c_Quais_dessas_responsabilidades_fazem_parte_da_sua_rotina_atual_de_trabalho_como_gestor,P3_c_1_Pensar_na_visao_de_longo_prazo_de_dados_da_empresa_e_fortalecimento_da_cultura_analitica_da_companhia,P3_c_2_Organizacao_de_treinamentos_e_iniciativas_com_o_objetivo_de_aumentar_a_maturidade_analitica_das_areas_de_negocios,P3_c_3_Atracao_selecao_e_contratacao_de_talentos_para_o_time_de_dados,P3_c_4_Decisao_sobre_contratacao_de_ferramentas_e_tecnologias_relacionadas_a_dados,P3_c_5_Sou_gestor_da_equipe_responsavel_pela_engenharia_de_dados_e_por_manter_o_Data_Lake_da_empresa_como_fonte_unica_dos_dados_garantindo_a_qualidade_e_confiabilidade_da_informacao,P3_c_

## Etapa 3 - Camada Silver

#### Etapas
Aqui dividiremos o trabalho em algumas etapas:
- Tratar as colunas, ajustar os nomes para ficarem padronizados
- Tratar os dados dentro, remover nulos
- Selecionar a tipagem de cada coluna, descrição
- Ajustar os valores padronizando em algumas colunas

Os dados são normalizados em seus nomes de colunas para garantir compatibilidade com o Databricks.

In [0]:
from pyspark.sql.functions import col, current_date
# Caminho da Silver
silver_path = "dbfs:/mnt/silver/state_of_data_2023"

# Criação da camada silver
spark.sql("CREATE SCHEMA IF NOT EXISTS silver");

# Colunas da Bronze (nomes atuais da tabela)
selected_columns = [
    "P0_id", "P1_a_Idade", "P1_a_1_Faixa_idade", "P1_b_Genero", "P1_c_Cor_raca_etnia",
    "P1_d_PCD", "P1_e_experiencia_profissional_prejudicada",
    "P1_e_1_Nao_acredito_que_minha_experiencia_profissional_seja_afetada",
    "P1_e_2_Experiencia_prejudicada_devido_a_minha_Cor_Raca_Etnia",
    "P1_e_3_Experiencia_prejudicada_devido_a_minha_identidade_de_genero",
    "P1_e_4_Experiencia_prejudicada_devido_ao_fato_de_ser_PCD",
    "P1_f_aspectos_prejudicados", "P1_f_1_Quantidade_de_oportunidades_de_emprego_vagas_recebidas",
    "P1_f_2_Senioridade_das_vagas_recebidas_em_relacao_a_sua_experiencia",
    "P1_f_3_Aprovacao_em_processos_seletivos_entrevistas",
    "P1_f_4_Oportunidades_de_progressao_de_carreira",
    "P1_f_5_Velocidade_de_progressao_de_carreira",
    "P1_f_6_Nivel_de_cobranca_no_trabalho_Stress_no_trabalho",
    "P1_f_7_Atencao_dada_diante_das_minhas_opinioes_e_ideias",
    "P1_f_8_Relacao_com_outros_membros_da_empresa_em_momentos_de_trabalho",
    "P1_f_9_Relacao_com_outros_membros_da_empresa_em_momentos_de_integracao_e_outros_momentos_fora_do_trabalho",
    "P1_g_vive_no_brasil", "P1_i_Estado_onde_mora", "P1_i_1_uf_onde_mora", "P1_i_2_Regiao_onde_mora",
    "P1_j_Mudou_de_Estado", "P1_k_Regiao_de_origem", "P1_l_Nivel_de_Ensino", "P1_m_Area_de_Formacao",
    "P2_G_NIVEL", "P2_H_FAIXA_SALARIAL", "P2_I_QUANTO_TEMPO_DE_EXPERIENCIA_NA_AREA_DE_DADOS_VOCE_TEM"
]

# Novos nomes desejados (para a camada Silver)
novos_nomes = [
    "id", "idade", "faixa_idade", "genero", "cor_raca_etnia", "pcd", "exp_profissional_prejudicada",
    "nao_acredita_afetado", "exp_prejud_cor_raca", "exp_prejud_genero", "exp_prejud_pcd",
    "aspectos_prejudicados", "qtde_oportunidades", "senioridade_vagas", "aprovacao_seletivos",
    "prog_carreira", "vel_prog_carreira", "nivel_cobranca", "atencao_ideias", "relacao_trabalho",
    "relacao_integracao", "vive_no_brasil", "estado_mora", "uf_mora", "regiao_mora", "mudou_estado",
    "regiao_origem", "nivel_ensino", "area_formacao", "senioridade", "faixa_salarial", "tempo_experiencia"
]

# Mapeamento {coluna_bronze: coluna_silver}
colunas_renomeadas = dict(zip(selected_columns, novos_nomes))

# Leitura da Bronze
df_bronze = spark.table("bronze.state_of_data_2023")

# Selecionar e renomear colunas
df_silver = df_bronze.select([col(old).alias(new) for old, new in colunas_renomeadas.items()])

# Adiciona coluna de data da silver, data de quando foi alterado da bronze>silver
df_silver = df_silver.withColumn("dt_silver", current_date())

In [0]:
# Caminho onde os dados da Silver serão salvos
silver_path = "dbfs:/mnt/silver/state_of_data_2023"

# Escrevendo no Delta Lake
df_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(silver_path)

# Cria o schema silver se ainda não existir
spark.sql("DROP TABLE IF EXISTS silver.state_of_data")

# Apagar a tabela lógica e os dados físicos (evitando problema de execução)
spark.sql("DROP TABLE IF EXISTS silver.state_of_data")
dbutils.fs.rm("dbfs:/mnt/silver/state_of_data_2023", recurse=True)

Out[19]: True

In [0]:
# Criação da tabela já com os comentários e tipagem, para facilitar documentação e identificação
spark.sql("""
CREATE OR REPLACE TABLE silver.state_of_data (
  id STRING COMMENT 'Identificador único do respondente',
  idade INT COMMENT 'Idade do respondente',
  faixa_idade STRING COMMENT 'Faixa etária do respondente',
  genero STRING COMMENT 'Identidade de gênero',
  cor_raca_etnia STRING COMMENT 'Cor, raça ou etnia',
  pcd STRING COMMENT 'Pessoa com deficiência (PCD)',
  exp_profissional_prejudicada STRING COMMENT 'Declaração de impacto na experiência profissional',
  nao_acredita_afetado INT COMMENT 'Não acredita que sua experiência tenha sido afetada',
  exp_prejud_cor_raca INT COMMENT 'Experiência prejudicada por cor, raça ou etnia',
  exp_prejud_genero INT COMMENT 'Experiência prejudicada por identidade de gênero',
  exp_prejud_pcd INT COMMENT 'Experiência prejudicada por ser PCD',
  aspectos_prejudicados STRING COMMENT 'Aspectos considerados prejudicados na carreira',
  qtde_oportunidades STRING COMMENT 'Quantidade de oportunidades de emprego recebidas',
  senioridade_vagas STRING COMMENT 'Senioridade das vagas recebidas',
  aprovacao_seletivos INT COMMENT 'Aprovação em processos seletivos/entrevistas',
  prog_carreira INT COMMENT 'Oportunidades de progressão de carreira',
  vel_prog_carreira INT COMMENT 'Velocidade da progressão de carreira',
  nivel_cobranca INT COMMENT 'Nível de cobrança ou estresse no trabalho',
  atencao_ideias INT COMMENT 'Atenção dada às opiniões e ideias',
  relacao_trabalho INT COMMENT 'Relação com colegas no ambiente de trabalho',
  relacao_integracao INT COMMENT 'Relação com colegas fora do trabalho',
  vive_no_brasil INT COMMENT 'Indica se reside no Brasil',
  estado_mora STRING COMMENT 'Estado onde reside atualmente',
  uf_mora STRING COMMENT 'UF onde reside',
  regiao_mora STRING COMMENT 'Região onde reside',
  mudou_estado STRING COMMENT 'Indica se mudou de estado',
  regiao_origem STRING COMMENT 'Região de origem',
  nivel_ensino STRING COMMENT 'Nível de escolaridade',
  area_formacao STRING COMMENT 'Área de formação acadêmica',
  senioridade STRING COMMENT 'Senioridade',
  faixa_salarial STRING COMMENT 'Faixa salarial atual',
  tempo_experiencia STRING COMMENT 'Tempo de experiencia na area de dados',
  dt_silver DATE COMMENT 'Data de ingestão na camada silver'
)
USING DELTA
LOCATION 'dbfs:/mnt/silver/state_of_data_2023'
""")

# Reescrever os dados com o DataFrame
df_silver.write.format("delta").mode("overwrite").save("dbfs:/mnt/silver/state_of_data_2023")


In [0]:
from pyspark.sql.functions import col, when, regexp_replace

# Carrega a silver atual
df = spark.table("silver.state_of_data")

# Criando um de-para
de_para_exp = {
    'Menos de 1 ano': '0-1',
    'de 1 a 2 anos': '1-2',
    'de 3 a 4 anos': '3-4',
    'de 4 a 6 anos': '4-6',
    'de 5 a 6 anos': '5-6',
    'de 7 a 10 anos': '7-10',
    'Mais de 10 anos': '10+',
    'Não tenho experiência na área de dados': '0'
}

# Cria coluna com os valores válidos apenas
df = df.withColumn("tempo_experiencia_tratada",
                   when(col("tempo_experiencia").isin(list(de_para_exp.keys())), col("tempo_experiencia")))

# Aplica o de-para
for original, novo in de_para_exp.items():
    df = df.withColumn("tempo_experiencia_tratada",
                       when(col("tempo_experiencia") == original, novo).otherwise(col("tempo_experiencia_tratada")))

# Remove registros não mapeados
df = df.filter(col("tempo_experiencia_tratada").isNotNull())

In [0]:
# Filtrando valores nulos e Outra Opção, para manter a coluna com informações relevantes
df = df.filter(
    col("faixa_salarial").isNotNull() &
    (~col("faixa_salarial").rlike("Outra Opção"))
)

# Removendo caracteres e palavras que não compoem a faixa_salarial, mantendo apenas os valores
df = (
    df
    .withColumn("faixa_salarial_tratada", regexp_replace("faixa_salarial", "\\.", ""))
    .withColumn("faixa_salarial_tratada", regexp_replace("faixa_salarial_tratada", ",", ""))
    .withColumn("faixa_salarial_tratada", regexp_replace("faixa_salarial_tratada", "R\\$", ""))
    .withColumn("faixa_salarial_tratada", regexp_replace("faixa_salarial_tratada", "/mês", ""))
    .withColumn("faixa_salarial_tratada", regexp_replace("faixa_salarial_tratada", "de ", ""))
    .withColumn("faixa_salarial_tratada", regexp_replace("faixa_salarial_tratada", " a ", " - "))
    .withColumn("faixa_salarial_tratada", regexp_replace("faixa_salarial_tratada", "Menos de ", "0 - "))
    .withColumn("faixa_salarial_tratada", regexp_replace("faixa_salarial_tratada", "Acima de ", "40001 - "))
    
)

# Aplicando tratamento da faixa_salarial no dataframe
df_final = (
    df
    .drop("tempo_experiencia", "faixa_salarial")
    .withColumnRenamed("tempo_experiencia_tratada", "tempo_experiencia")
    .withColumnRenamed("faixa_salarial_tratada", "faixa_salarial")
)



In [0]:
# Escrevendo na tabela silver
df_final.write.format("delta").mode("overwrite").saveAsTable("silver.state_of_data")


In [0]:
%sql
select id, idade, faixa_idade, faixa_salarial, tempo_experiencia, * from silver.state_of_data limit 10

id,idade,faixa_idade,faixa_salarial,tempo_experiencia,id,idade,faixa_idade,genero,cor_raca_etnia,pcd,exp_profissional_prejudicada,nao_acredita_afetado,exp_prejud_cor_raca,exp_prejud_genero,exp_prejud_pcd,aspectos_prejudicados,qtde_oportunidades,senioridade_vagas,aprovacao_seletivos,prog_carreira,vel_prog_carreira,nivel_cobranca,atencao_ideias,relacao_trabalho,relacao_integracao,vive_no_brasil,estado_mora,uf_mora,regiao_mora,mudou_estado,regiao_origem,nivel_ensino,area_formacao,senioridade,faixa_salarial,tempo_experiencia,dt_silver
001b2d1qtli8t9z7oqgdhj001b2d4i0g,31,30-34,12001 - 16000,3-4,001b2d1qtli8t9z7oqgdhj001b2d4i0g,31,30-34,Masculino,Branca,Não,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,Minas Gerais (MG),MG,Sudeste,1,null,Doutorado ou Phd,Computação / Engenharia de Software / Sistemas de Informação/ TI,Sênior,12001 - 16000,3-4,2025-04-12
0026aa3fwd78u0026asg7456tfkjg2cs,30,30-34,6001 - 8000,3-4,0026aa3fwd78u0026asg7456tfkjg2cs,30,30-34,Masculino,Branca,Não,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,Espírito Santo (ES),ES,Sudeste,1,null,Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas de Informação/ TI,Pleno,6001 - 8000,3-4,2025-04-12
00r21rb9pusd1b0v7ew00r21rw3dy69w,37,35-39,6001 - 8000,1-2,00r21rb9pusd1b0v7ew00r21rw3dy69w,37,35-39,Feminino,Amarela,Não,"Sim, acredito que a minha a experiência profissional seja afetada devido a minha identidade de gênero",0,0,1,0,"Atenção dada pelas pessoas diante das minhas opiniões e ideias, Aprovação em processos seletivos/entrevistas, Quantidade de oportunidades de emprego/vagas recebidas",1,0,1,0,0,0,1,0,0,1,São Paulo (SP),SP,Sudeste,1,null,Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas de Informação/ TI,Júnior,6001 - 8000,1-2,2025-04-12
00v0az4g792svil00vn6y1kfm9hq8vy9,34,30-34,4001 - 6000,0-1,00v0az4g792svil00vn6y1kfm9hq8vy9,34,30-34,Masculino,Branca,Não,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,Minas Gerais (MG),MG,Sudeste,1,null,Doutorado ou Phd,Economia/ Administração / Contabilidade / Finanças/ Negócios,Júnior,4001 - 6000,0-1,2025-04-12
015qqbrh4rr015qje9gvkbsb7ti1n5df,30,30-34,8001 - 12000,1-2,015qqbrh4rr015qje9gvkbsb7ti1n5df,30,30-34,Masculino,Branca,Não,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,Rio de Janeiro (RJ),RJ,Sudeste,1,null,Estudante de Graduação,Computação / Engenharia de Software / Sistemas de Informação/ TI,Pleno,8001 - 12000,1-2,2025-04-12
01810y9hweneb018to8t1p8ubyn1o04m,24,22-24,4001 - 6000,3-4,01810y9hweneb018to8t1p8ubyn1o04m,24,22-24,Masculino,Branca,Não,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,São Paulo (SP),SP,Sudeste,1,null,Graduação/Bacharelado,Economia/ Administração / Contabilidade / Finanças/ Negócios,Pleno,4001 - 6000,3-4,2025-04-12
01k1i8w42ljmgee01k1eay1n1283ot23,35,35-39,20001 - 25000,7-10,01k1i8w42ljmgee01k1eay1n1283ot23,35,35-39,Masculino,Branca,Não,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,São Paulo (SP),SP,Sudeste,0,São Paulo (SP),Pós-graduação,Computação / Engenharia de Software / Sistemas de Informação/ TI,null,20001 - 25000,7-10,2025-04-12
01zusm1faol2k2601zlthh05cj7iqyit,25,25-29,1001 - 2000,1-2,01zusm1faol2k2601zlthh05cj7iqyit,25,25-29,Masculino,Preta,Sim,"Sim, acredito que a minha a experiência profissional seja afetada devido a minha Cor/Raça/Etnia, Sim, acredito que minha experiência profissional seja afetada devido ao fato de ser PCD",0,1,0,1,"Quantidade de oportunidades de emprego/vagas recebidas, Aprovação em processos seletivos/entrevistas, Nível de cobrança no trabalho / Stress no trabalho, Relação com outros membros da empresa, em momentos de trabalho",1,0,1,0,0,1,0,1,0,1,Sergipe (SE),SE,Nordeste,1,null,Estudante de Graduação,Computação / Engenharia de Software / Sistemas de Informação/ TI,Júnior,1001 - 2000,1-2,2025-04-12
02yuxvtb6698rqwq8w02yuza594l7cut,32,30-34,3001 - 4000,1-2,02y

## Etapa 4 - Visões Analiticas - Camada Gold


In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS gold");

### Analise 1 - Faixa Salarial por Experiência (gold.faixa_salarial_por_experiencia)
Objetivo: Relacionar o tempo de experiência com a faixa salarial informada.

Analise:

Profissionais com mais de 10 anos de experiência tendem a estar nas faixas salariais mais altas (acima de R$ 16.000).

Já iniciantes (até 2 anos) concentram-se majoritariamente nas faixas de R$ 1.000 a R$ 4.000.

Existe uma correlação positiva entre tempo de experiência e salário, embora com dispersão em alguns intervalos.

In [0]:
%sql
CREATE OR REPLACE TABLE gold.faixa_salarial_por_experiencia AS -- criação da tabela gold
SELECT
  tempo_experiencia,
  faixa_salarial,
  COUNT(*) AS total_pessoas
FROM silver.state_of_data
WHERE faixa_salarial IS NOT NULL AND tempo_experiencia IS NOT NULL
GROUP BY tempo_experiencia, faixa_salarial;

-- Visualização de exemplo
SELECT * FROM gold.faixa_salarial_por_experiencia
WHERE total_pessoas > 10
ORDER BY total_pessoas DESC;


tempo_experiencia,faixa_salarial,total_pessoas
3-4,8001 - 12000,357
1-2,4001 - 6000,351
3-4,6001 - 8000,225
1-2,6001 - 8000,217
3-4,4001 - 6000,178
1-2,8001 - 12000,175
1-2,3001 - 4000,173
3-4,12001 - 16000,144
4-6,8001 - 12000,137
1-2,2001 - 3000,127


### Analise 2 - Salário por Gênero e Cor/Raça (gold.salario_genero_cor_raca)
Objetivo: Avaliar se há diferenças salariais por gênero e etnia.

Analise:

Há maior concentração de homens brancos nas faixas salariais mais altas.

Mulheres e pessoas negras ou pardas estão sub-representadas nas faixas salariais superiores, mesmo com senioridade compatível.

Indício de desigualdade estrutural entre grupos.

In [0]:
%sql
CREATE OR REPLACE TABLE gold.salario_genero_cor_raca AS -- criação da tabela gold
SELECT
  genero,
  cor_raca_etnia,
  faixa_salarial,
  COUNT(*) AS total_pessoas
FROM silver.state_of_data
WHERE genero IS NOT NULL AND cor_raca_etnia IS NOT NULL AND faixa_salarial IS NOT NULL
GROUP BY genero, cor_raca_etnia, faixa_salarial;

-- Visualização de exemplo
SELECT * FROM gold.salario_genero_cor_raca
WHERE total_pessoas > 10
ORDER BY total_pessoas DESC;


genero,cor_raca_etnia,faixa_salarial,total_pessoas
Masculino,Branca,8001 - 12000,506
Masculino,Branca,4001 - 6000,351
Masculino,Branca,12001 - 16000,341
Masculino,Branca,6001 - 8000,293
Masculino,Parda,8001 - 12000,202
Masculino,Branca,16001 - 20000,194
Feminino,Branca,8001 - 12000,160
Masculino,Parda,4001 - 6000,159
Masculino,Branca,3001 - 4000,149
Masculino,Branca,20001 - 25000,125


### Analise 3 - Nível de Ensino por Senioridade (gold.nivel_ensino_senioridade)
Objetivo: Relacionar escolaridade com o nível de senioridade.

Analise:

Senioridade Pleno e Sênior estão mais associadas a Ensino Superior completo ou pós-graduação.

Profissionais Júnior aparecem em todos os níveis de ensino, com destaque para cursos técnicos ou ensino superior incompleto.

A formação acadêmica não é a única determinante da senioridade, mas há tendência.

In [0]:
%sql
CREATE OR REPLACE TABLE gold.nivel_ensino_senioridade AS  -- criação da tabela gold
SELECT
  nivel_ensino,
  senioridade,
  COUNT(*) AS total_pessoas
FROM silver.state_of_data
WHERE senioridade IS NOT NULL AND nivel_ensino IS NOT NULL
GROUP BY nivel_ensino, senioridade;

-- Visualização de exemplo
SELECT * FROM gold.nivel_ensino_senioridade
WHERE total_pessoas > 10
ORDER BY total_pessoas DESC;


nivel_ensino,senioridade,total_pessoas
Pós-graduação,Sênior,614
Graduação/Bacharelado,Pleno,573
Graduação/Bacharelado,Sênior,452
Pós-graduação,Pleno,446
Graduação/Bacharelado,Júnior,421
Estudante de Graduação,Júnior,324
Mestrado,Sênior,216
Pós-graduação,Júnior,203
Mestrado,Pleno,172
Estudante de Graduação,Pleno,131


### Analise 4 - Mudança de Estado por Faixa Salarial (gold.mudanca_estado_salario)
Objetivo: Entender se a mobilidade geográfica está relacionada à remuneração.

Analise:

Profissionais que mudaram de estado tendem a estar em faixas salariais mais altas.

Isso sugere que a mobilidade pode estar associada a melhores oportunidades (promoções, novos empregos).

Já quem permaneceu no mesmo estado está mais distribuído nas faixas inferiores.

In [0]:
%sql
CREATE OR REPLACE TABLE gold.mudanca_estado_salario AS  -- criação da tabela gold
SELECT
  mudou_estado = 'Sim' AS mudou_estado,
  faixa_salarial,
  COUNT(*) AS total_pessoas
FROM silver.state_of_data
WHERE mudou_estado IS NOT NULL AND faixa_salarial IS NOT NULL
GROUP BY mudou_estado, faixa_salarial;


-- Visualização de exemplo
SELECT * FROM gold.mudanca_estado_salario
WHERE total_pessoas > 5
ORDER BY total_pessoas DESC;


mudou_estado,faixa_salarial,total_pessoas
false,8001 - 12000,820
false,4001 - 6000,603
false,6001 - 8000,498
false,12001 - 16000,479
false,3001 - 4000,290
false,16001 - 20000,242
false,2001 - 3000,242
false,8001 - 12000,191
false,1001 - 2000,172
false,12001 - 16000,158


### Analise 5 - Discriminação na Carreira (gold.discriminacao_carreira)
Objetivo: Analisar se a percepção de discriminação afeta salário e senioridade.

Analise:

Pessoas que afirmaram ter sofrido discriminação por cor/raça, gênero ou deficiência concentram-se nas faixas mais baixas de salário.

A percepção de carreira prejudicada também aparece entre pessoas com senioridade alta, o que reforça um sentimento de barreira invisível à ascensão profissional.

Grupos minorizados enfrentam desigualdade mesmo com qualificação.

In [0]:
%sql
CREATE OR REPLACE TABLE gold.discriminacao_carreira AS -- criação da tabela gold
SELECT
  exp_profissional_prejudicada,
  exp_prejud_cor_raca,
  exp_prejud_genero,
  exp_prejud_pcd,
  senioridade,
  faixa_salarial,
  COUNT(*) AS total_pessoas
FROM silver.state_of_data
WHERE exp_profissional_prejudicada IS NOT NULL
GROUP BY
  exp_profissional_prejudicada,
  exp_prejud_cor_raca,
  exp_prejud_genero,
  exp_prejud_pcd,
  senioridade,
  faixa_salarial;

-- Visualização de exemplo
SELECT * FROM gold.discriminacao_carreira
WHERE total_pessoas > 5
ORDER BY total_pessoas DESC;


exp_profissional_prejudicada,exp_prejud_cor_raca,exp_prejud_genero,exp_prejud_pcd,senioridade,faixa_salarial,total_pessoas
Não acredito que minha experiência profissional seja afetada devido a esses fatores,0,0,0,Sênior,8001 - 12000,131
Não acredito que minha experiência profissional seja afetada devido a esses fatores,0,0,0,Pleno,8001 - 12000,113
Não acredito que minha experiência profissional seja afetada devido a esses fatores,0,0,0,Pleno,4001 - 6000,100
Não acredito que minha experiência profissional seja afetada devido a esses fatores,0,0,0,Júnior,4001 - 6000,99
Não acredito que minha experiência profissional seja afetada devido a esses fatores,0,0,0,Pleno,6001 - 8000,98
Não acredito que minha experiência profissional seja afetada devido a esses fatores,0,0,0,Sênior,12001 - 16000,91
"Sim, acredito que a minha a experiência profissional seja afetada devido a minha identidade de gênero",0,1,0,Sênior,8001 - 12000,69
Não acredito que minha experiência profissional seja afetada devido a esses fatores,0,0,0,Júnior,3001 - 4000,69
Não acredito que minha experiência profissional seja afetada devido a esses fatores,0,0,0,Júnior,1001 - 2000,61
Não acredito que minha experiência profissional seja afetada devido a esses fatores,0,0,0,Júnior,2001 - 3000,59


### Analise 6 - Média Salarial por Senioridade e Experiência (gold.media_salarial_senioridade_experiencia)
Objetivo: Calcular a média salarial para cruzamento de tempo de experiência com senioridade.

Analise:

A média salarial cresce linearmente com o tempo de experiência e nível de senioridade.

Há sobreposição entre Pleno com até 4 anos de experiência e Sênior com pouca experiência, indicando possíveis distorções de cargo.

Profissionais Júnior com mais tempo de experiência também aparecem, sugerindo retenção em cargos abaixo da experiência real.

In [0]:
%sql
-- Utilização de regex para segregar faixa_salarial em salario minimo e maximo, para termos uma media do salario
CREATE OR REPLACE TABLE gold.media_salarial_senioridade_experiencia AS  -- criação da tabela gold
SELECT
  senioridade,
  tempo_experiencia,
  CAST(REGEXP_EXTRACT(REGEXP_REPLACE(faixa_salarial, r'\s+', ''), r'^(\d+)-') AS INT) AS salario_minimo,
  CAST(REGEXP_EXTRACT(REGEXP_REPLACE(faixa_salarial, r'\s+', ''), r'-(\d+)$') AS INT) AS salario_maximo,
  ROUND((
    CAST(REGEXP_EXTRACT(REGEXP_REPLACE(faixa_salarial, r'\s+', ''), r'^(\d+)-') AS DOUBLE) +
    CAST(REGEXP_EXTRACT(REGEXP_REPLACE(faixa_salarial, r'\s+', ''), r'-(\d+)$') AS DOUBLE)
  ) / 2, 2) AS salario_medio,
  COUNT(*) AS total_pessoas
FROM silver.state_of_data
WHERE faixa_salarial NOT IN ('Menos  1000', 'Acima  40001')
  AND senioridade IS NOT NULL
  AND tempo_experiencia IS NOT NULL
GROUP BY senioridade, tempo_experiencia, faixa_salarial;

-- Visualização de exemplo
SELECT * FROM gold.media_salarial_senioridade_experiencia
WHERE total_pessoas > 10
ORDER BY salario_medio DESC;


senioridade,tempo_experiencia,salario_minimo,salario_maximo,salario_medio,total_pessoas
Sênior,10+,25001,30000,27500.5,12
Sênior,7-10,20001,25000,22500.5,16
Sênior,4-6,20001,25000,22500.5,11
Sênior,10+,20001,25000,22500.5,22
Sênior,5-6,16001,20000,18000.5,20
Sênior,4-6,16001,20000,18000.5,32
Sênior,7-10,16001,20000,18000.5,28
Sênior,3-4,16001,20000,18000.5,20
Sênior,10+,16001,20000,18000.5,34
Sênior,10+,12001,16000,14000.5,71
